In [2]:
import pandas as pd
import re
from unidecode import unidecode
from collections import defaultdict

In [3]:
# 1. Cargar los datos
df = pd.read_excel("DatosTecnicaTurboshop.xlsx", sheet_name="F2")
df.head()

,Codigo,Nombre,Aplicaciones
0,9075788,CALIPER DELANTERO IZQUIERDO (NORTARGE),SAIL 1.4 2011/2018
1,9075789,CALIPER DELANTERO DERECHO (NORTARGE),SAIL 1.4 2011/2018
2,9075789,CALIPER DELANTERO DERECHO (UXCLENT),SAIL 1.4 2011/2018
3,9075910,FILTRO POLEN (KENDALL),SAIL 1.4 1.5 2010/2021
4,9114603,SENSOR VELOCIDAD (NETMOTORS),CORSA 1.4 1.6 1993/1998-ASTRA 1.4 1.6 1992/199...


In [4]:

def extraer_autos_preciso(df_repuestos):
    # Diccionario de marcas con modelos específicos
    marcas_modelos = {
        # Chevrolet/Opel
        'CORSA': ['CHEVROLET', 'OPEL'],
        'ASTRA': ['CHEVROLET', 'OPEL'],
        'VECTRA': ['CHEVROLET', 'OPEL'],
        'ZAFIRA': ['CHEVROLET', 'OPEL'],
        'COMBO': ['CHEVROLET', 'OPEL'],
        
        # Chevrolet exclusivos
        'SAIL': ['CHEVROLET'],
        'SPARK': ['CHEVROLET'],
        'AVEO': ['CHEVROLET'],
        'OPTRA': ['CHEVROLET'],
        'CAPTIVA': ['CHEVROLET'],
        'MONTANA': ['CHEVROLET'],
        'EPICA': ['CHEVROLET'],
        
        # Otras marcas
        'ACCENT': ['HYUNDAI'],
        'EXCEL': ['HYUNDAI'],
        'SONATA': ['HYUNDAI'],
        'ELANTRA': ['HYUNDAI'],
        'TUCSON': ['HYUNDAI'],
        'GETZ': ['HYUNDAI'],
        'SANTA FE': ['HYUNDAI'],
        
        # Daewoo
        'ESPERO': ['DAEWOO'],
        'HEAVEN': ['DAEWOO'],
        'LANOS': ['DAEWOO'],
        'RACER': ['DAEWOO'],
        'NEXIA': ['DAEWOO'],
    }

    registros = []
    
    for aplicaciones in df_repuestos['Aplicaciones'].dropna():
        aplicaciones = str(aplicaciones)
        
        # Dividir por guiones manteniendo el modelo y sus especificaciones
        segmentos = re.split(r'(?<=\d)\s*-\s*', aplicaciones)
        
        for segmento in segmentos:
            segmento = segmento.strip()
            if not segmento:
                continue
            
            # Extraer el modelo base (primera palabra)
            modelo_match = re.match(r'^([A-Z][A-Za-z]+)', segmento)
            if not modelo_match:
                continue
                
            modelo_base = modelo_match.group(1)
            
            if modelo_base in marcas_modelos:
                # Extraer solo cilindradas (formato número.número)
                cilindradas = re.findall(r'\b(\d\.\d)\b', segmento)
                cilindradas = list(set(cilindradas))  # Eliminar duplicados
                
                # Extraer años (rangos o individuales)
                años = re.findall(r'\b(\d{4})[/-](\d{4})\b|\b(\d{4})\b', segmento)
                
                # Procesar años
                años_procesados = []
                for año in años:
                    if año[2]:  # Año individual (tercer grupo)
                        años_procesados.append(int(año[2]))
                    else:  # Rango de años (primer y segundo grupo)
                        inicio = int(año[0])
                        fin = int(año[1])
                        años_procesados.extend(range(inicio, fin + 1))
                
                # Si no hay años, usar ND
                if not años_procesados:
                    años_procesados = ['ND']
                
                # Si no hay cilindradas, usar ND
                if not cilindradas:
                    cilindradas = ['ND']
                
                # Generar todas las combinaciones
                for marca in marcas_modelos[modelo_base]:
                    for cilindrada in cilindradas:
                        for año in años_procesados:
                            registros.append({
                                'MARCA': marca,
                                'MODELO': modelo_base,
                                'CILINDRADA': cilindrada,
                                'AÑO': año,
                                'APLICACION_ORIGINAL': aplicaciones
                            })
    
    # Crear DataFrame y limpiar
    df_autos = pd.DataFrame(registros)
    df_autos = df_autos.drop_duplicates()
    
    return df_autos.sort_values(['MARCA', 'MODELO', 'CILINDRADA', 'AÑO']).reset_index(drop=True)

In [5]:
# Uso
df_repuestos = pd.read_excel("DatosTecnicaTurboshop.xlsx", sheet_name="F2")
df_autos = extraer_autos_preciso(df_repuestos)
df_autos[df_autos['MODELO'] == 'ASTRA'].head(5)

,MARCA,MODELO,CILINDRADA,AÑO,APLICACION_ORIGINAL
0,CHEVROLET,ASTRA,1.4,1992,CORSA 1.4 1.6 1993/1998-ASTRA 1.4 1.6 1992/199...
1,CHEVROLET,ASTRA,1.4,1993,CORSA 1.4 1.6 1993/1998-ASTRA 1.4 1.6 1992/199...
2,CHEVROLET,ASTRA,1.4,1994,CORSA 1.4 1.6 1993/1998-ASTRA 1.4 1.6 1992/199...
3,CHEVROLET,ASTRA,1.4,1995,CORSA 1.4 1.6 1993/1998-ASTRA 1.4 1.6 1992/199...
4,CHEVROLET,ASTRA,1.4,1995,CORSA 1.0 1.4 1.6 1993/2010-COMBO 1.4 1995/19...


In [6]:
df_3 = df_autos.drop('APLICACION_ORIGINAL', axis=1).copy()
df_3

,MARCA,MODELO,CILINDRADA,AÑO
0,CHEVROLET,ASTRA,1.4,1992
1,CHEVROLET,ASTRA,1.4,1993
2,CHEVROLET,ASTRA,1.4,1994
3,CHEVROLET,ASTRA,1.4,1995
4,CHEVROLET,ASTRA,1.4,1995
...,...,...,...,...
930,OPEL,ZAFIRA,ND,2002
931,OPEL,ZAFIRA,ND,2003
932,OPEL,ZAFIRA,ND,2004
933,OPEL,ZAFIRA,ND,2005


In [7]:
df_3 = df_3.drop_duplicates().copy()
df_3

,MARCA,MODELO,CILINDRADA,AÑO
0,CHEVROLET,ASTRA,1.4,1992
1,CHEVROLET,ASTRA,1.4,1993
2,CHEVROLET,ASTRA,1.4,1994
3,CHEVROLET,ASTRA,1.4,1995
5,CHEVROLET,ASTRA,1.4,1996
...,...,...,...,...
930,OPEL,ZAFIRA,ND,2002
931,OPEL,ZAFIRA,ND,2003
932,OPEL,ZAFIRA,ND,2004
933,OPEL,ZAFIRA,ND,2005


In [8]:
# Paso 1: Crear una columna temporal que indique si el grupo tiene años no-ND
df_3['tiene_año_especifico'] = df_3.groupby(['MARCA', 'MODELO', 'CILINDRADA'])['AÑO'].transform(
    lambda x: any(x != 'ND')
).copy()

# Paso 2: Filtrar (eliminar ND solo si existe otro registro con año específico)
df_3 = df_3[~((df_3['AÑO'] == 'ND') & (df_3['tiene_año_especifico']))].copy()

# Paso 3: Eliminar columna temporal
df_3 = df_3.drop(columns=['tiene_año_especifico']).copy()

In [9]:
# Paso 1: Crear columna temporal que indique si el grupo tiene cilindradas definidas
df_3['tiene_cilindrada_especifica'] = df_3.groupby(['MARCA', 'MODELO', 'AÑO'])['CILINDRADA'].transform(
    lambda x: any(x != 'ND')  # True si hay al menos una cilindrada definida en el grupo
)

# Paso 2: Filtrar (eliminar "ND" en CILINDRADA solo si existe otra fila con cilindrada definida)
df_3 = df_3[~((df_3['CILINDRADA'] == 'ND') & (df_3['tiene_cilindrada_especifica']))]

# Paso 3: Eliminar columna temporal
df_3 = df_3.drop(columns=['tiene_cilindrada_especifica'])

In [10]:
df_3 = df_3.reset_index(drop=True)
#df_3[df_3['MODELO'] == 'SAIL']
df_3

,MARCA,MODELO,CILINDRADA,AÑO
0,CHEVROLET,ASTRA,1.4,1992
1,CHEVROLET,ASTRA,1.4,1993
2,CHEVROLET,ASTRA,1.4,1994
3,CHEVROLET,ASTRA,1.4,1995
4,CHEVROLET,ASTRA,1.4,1996
...,...,...,...,...
566,OPEL,ZAFIRA,2.0,2005
567,OPEL,ZAFIRA,2.0,2006
568,OPEL,ZAFIRA,ND,1999
569,OPEL,ZAFIRA,ND,2000


In [11]:
df_3.to_csv("data_limpia/datos_limpios_F2.csv", index=False, encoding='utf-8-sig')